<a href="https://colab.research.google.com/github/colleenrpy/MONAI_handson/blob/main/MONAI%20Datasets%20and%20Caching_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/Project-MONAI/MONAIBootcamp2021/blob/main/day1/monai.png?raw=1" style="width: 700px;"/>

MONAIハンズオンセミナーへようこそ! このノートブックでは、MONAI Caching, Datasets and Network オプションを紹介し、次にハンズオンとアーキテクチャ、そしてテストデータセットにSmartCacheをインプリメントするハンズオンを行います。

### Google Colabの使用

このノートブックにはMONAIをインストールするためのpipコマンドがあり、それ以降のノートブックには追加されます。

**Colabの実行に必要なパッケージ**。

colabノートブックの初回実行時にMONAIをインストールするために、以下のセルを実行します：


In [ ]:
!pip install -qU "monai[ignite, nibabel, torchvision, tqdm]==0.6.0"

**GPUサポートの有効化**

ColabでGPUリソースを使用するには、ランタイムをGPUに変更します。

1. 1. "Runtime "**メニューから、**"Change Runtime Type "**を選択します。
2. ドロップダウン・メニューから **"GPU "** を選択します。
3. **"SAVE "**をクリックします。

これにより、ノートブックがリセットされ、おそらくあなたがロボットであるかどうか尋ねられるでしょう（この手順は、あなたがそうでないと仮定しています）。

**!nvidia-smi**

をセルで実行すると、これがうまくいったかどうかが確認でき、あなたがどのようなハードウェアにアクセスできるかがわかります。  

In [ ]:
!nvidia-smi

# MONAIデータセット、キャッシング、ネットワークを理解
---

ユーザーは、望ましいモデル品質を達成するために、データに対して多くの（潜在的には数千の）エポックを用いてモデルを学習する必要があることがよくあります。PyTorchのネイティブ実装では、学習中にデータをロードし、エポックごとに同じ前処理を繰り返し実行することがありますが、これは特に医療画像のボリュームが大きい場合、時間がかかり、不要になることがあります。 データセットキャッシングを利用することで、システムがこのデータをロードして前処理を行う時間を短縮し、全体の学習時間を短縮することができます。

ネットワーク機能は、MONAIにとって重要な設計機会です。Pytorchは、ネットワークをどのように定義するかについて、非常に独創的な考え方を持っています。ネットワークを作成するための基本クラスとしてModuleを提供し、実装しなければならないいくつかのメソッドを提供します。しかし、ネットワークを初期化するための所定のパターンやヘルパー機能はあまりありません。

ヘルパー機能がないため、MONAI で新しいネットワークを構築するための有益な「ベストプラクティス」パターンを定義する余地が多く残されています。些細な、柔軟性に欠けるネットワークの実装は十分に簡単ですが、私たちはユーザーに、よく設計された柔軟なネットワークをはるかに容易に構築できるツールセットを提供し、構築したネットワークでそれを使用することを約束することによって、その価値を実証することができます。

## MONAIデータセット、キャッシング、ネットワーク

MONAI データセットとキャッシングオプションについてより深く理解するために、このガイドでは5つの重要な質問に答えることを手伝います：

1. **MONAI データセットとは何か？**
2. **データセットキャッシングとは何か、どのように使うのか？**
3. **MONAI が提供する一般的なデータセットには、どのようなものがありますか？**
4. **MONAI はどのようなネットワークとネットワークコンポーネントを提供していますか？**
5. **MONAI のレイヤーはどのように使用するのですか？**
6. **この柔軟なレイヤーを使って、どのようにネットワークを構築するのですか？**
7. **どのようなネットワークがMONAIに含まれていますか？**


まずは、dependenciesをインポートしてみましょう。

In [ ]:
import time
import torch

import monai
from monai.config import print_config
from monai.data import Dataset, DataLoader, CacheDataset, PersistentDataset, SmartCacheDataset
from monai.apps import DecathlonDataset
from monai.transforms import (
    MapTransform,
)

print_config()

MONAI version: 0.6.0
Numpy version: 1.20.3
Pytorch version: 1.8.1+cu102
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 0ad9e73639e30f4f1af5a1f4a45da9cb09930179

Optional dependencies:
Pytorch Ignite version: 0.4.4
Nibabel version: 3.2.1
scikit-image version: 0.18.1
Pillow version: 8.2.0
Tensorboard version: 2.5.0
gdown version: 3.13.0
TorchVision version: 0.9.1+cu102
ITK version: 5.1.2
tqdm version: 4.61.0
lmdb version: 1.2.1
psutil version: 5.8.0
pandas version: 1.2.4
einops version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## **1. What is a MONAI Dataset?**


MONAI データセットは、一般的なデータセットで、 __len__ プロパティ、 __getitem__ プロパティ、そしてデータサンプルを取得するときのオプションとして呼び出し可能なデータ変換を備えています。

まず、汎用データを初期化し、その汎用データで Dataset クラスを呼び出し、変換に None を指定します。

In [ ]:
items = [{"data": 4}, 
         {"data": 9}, 
         {"data": 3}, 
         {"data": 7}, 
         {"data": 1},
         {"data": 2},
         {"data": 5}]
dataset = monai.data.Dataset(items, transform=None)

print(f"Length of dataset is {len(dataset)}")
for item in dataset:
    print(item)

#### PyTorch DataLoaderとの互換性

MONAIの機能はPyTorch DataLoaderと互換性があるべきですが、標準のDataLoaderクラスでは実現できない、我々が重要だと考える追加機能があれば、自由にサブクラス化することが可能です。

In [ ]:
for item in torch.utils.data.DataLoader(dataset, batch_size=2):
    print(item)

### カスタマイズしたトランスフォームでアイテムをロードする

入力の `keys` に対応する値を二乗した値に置き換える、 `SquareIt` というカスタムトランスフォームを作成します。この例では、 `SquareIt(keys='data')` は `x['data']` の値に二乗変換を適用します。

In [ ]:
class SquareIt(MapTransform):
    def __init__(self, keys):
        MapTransform.__init__(self, keys)
        print(f"keys to square it: {self.keys}")
        
    def __call__(self, x):
        key = self.keys[0]
        data = x[key]
        output = {key: data ** 2}
        return output

square_dataset = Dataset(items, transform=SquareIt(keys='data'))
for item in square_dataset:
    print(item)

## **2. データセットキャッシュとは何ですか？**

 MONAIはマルチスレッド版の `CacheDataset` と `LMDBDataset` を提供し、変換チェーンにおける最初のランダム変換の前に中間結果を保存することで、学習中の変換ステップを高速化することができます。この機能を有効にすることで、Datasetsの実験では10倍の学習速度が得られる可能性があります。
 
<img src="https://github.com/Project-MONAI/MONAIBootcamp2021/blob/main/day1/cache_dataset.png?raw=1" style="width: 700px;"/>
 
データセットキャッシュの利点を示すために、低速な変換を行うデータセットを作成することにします。 そのために、各 `__call__` 関数の中で sleep 関数を呼び出すことにします。

In [ ]:
class SlowSquare(MapTransform):
    def __init__(self, keys):
        MapTransform.__init__(self, keys)
        print(f"keys to square it: {self.keys}")

    def __call__(self, x):
        time.sleep(1.0)
        output = {key: x[key] ** 2 for key in self.keys}
        return output

square_dataset = Dataset(items, transform=SlowSquare(keys='data'))

予想の通り、全項目に目を通すと7秒くらいかかりそうですね。

In [ ]:
%time for item in square_dataset: print(item)

このループを実行するたびに、すべてのアイテムを通過するのにおよそ7秒かかることになります。 これを100回繰り返すとすると、学習ループのロード時間が12分近く余計にかかることになります。 この時間を改善するために、キャッシュを利用する方法を考えてみましょう。

### Cache Dataset

[CacheDataset](https://docs.monai.io/en/latest/data.html?highlight=dataset#cachedataset)を使用した場合、キャッシュはオブジェクトの初回初期化時に行われるため、通常のデータセットに比べ初期化が遅くなります。

ランダムでない前処理変換の結果をキャッシュすることで、学習データのパイプラインを高速化することができます。要求されたデータがキャッシュにない場合、すべての変換が正常に実行されます。

In [ ]:
square_cached = CacheDataset(items, transform=SlowSquare(keys='data'))

しかし、初期化されたCacheDatasetから繰り返し項目を取得することは高速です。

In [ ]:
%timeit list(item for item in square_cached)

### Persistent Caching

[PersistentDataset](https://docs.monai.io/en/latest/data.html?highlight=dataset#persistentdataset) は、メモリより大きな辞書形式のデータを効率的に管理するために、事前に計算された値を永続的に保存することを可能にします。

非ランダム変換成分は、最初に使用するときに計算され、次に使用するときに素早く取り出せるように cache_dir に保存されます。

In [ ]:
square_persist = monai.data.PersistentDataset(items, transform=SlowSquare(keys='data'), cache_dir="my_cache")

In [ ]:
%time for item in square_persist: print(item)

PersistentDatasetの初期化時に、中間データを格納する場所をパラメータ "my_cache "で渡しました。以下、そのディレクトリを見ていきます。

In [ ]:
!ls my_cache

次のエポックでは、データセットに呼び出す際に、低速変換を呼び出さず、キャッシュされたデータを使用します。

In [ ]:
%timeit [item for item in square_persist]

新しいデータセットインスタンスは、キャッシュデータを利用することができます：

In [ ]:
square_persist_1 = monai.data.PersistentDataset(items, transform=SlowSquare(keys='data'), cache_dir="my_cache")
%timeit [item for item in square_persist_1]

#### キャッシュの動作
- また、[SmartCacheDataset](https://docs.monai.io/en/latest/data.html#monai.data.SmartCacheDataset)を使って、少ないメモリ消費でトランスフォームのレイテンシを隠蔽することもできます。
- データセットチュートリアルのノートブックには、MONAIにおけるキャッシュ機構の動作例と比較例があります: https://github.com/Project-MONAI/tutorials/blob/master/acceleration/dataset_type_performance.ipynb

<img src="https://github.com/Project-MONAI/MONAIBootcamp2021/blob/main/day1/datasets_speed.png?raw=1" style="width: 700px;"/>

## **3. MONAIで提供される共通データセットにはどのようなものがありますか？**

MONAIでは、医療分野の一般的な学習データをすぐに利用できるように、データ固有のDataset（MedNISTDataset、DecathlonDatasetなど）をいくつか提供しています。これらは、AWSストレージからのダウンロード、データファイルの抽出、変換による学習・評価項目の生成をサポートするものです。

DecathlonDataset](https://docs.monai.io/en/latest/data.html?highlight=dataset#decathlon-datalist)関数は、このノートブックを通して説明された機能を活用します。 これらのデータセットは、上記で説明したCacheDatasetの拡張版です。

In [ ]:
dataset = monai.apps.DecathlonDataset(root_dir="./", task="Task09_Spleen", section="training", download=True)

In [ ]:
print(dataset.get_properties("numTraining"))
print(dataset.get_properties("description"))

In [ ]:
print(dataset[0]['image'].shape)
print(dataset[0]['label'].shape)

## 4. **MONAI はどのようなネットワークとネットワークコンポーネントを提供しますか？**

MONAIは、Pytorchモジュール、Sequentialなどを直接継承したネットワークとそのコンポーネントの定義を提供します。これらの汎用ネットワークは、パラメータ化されたトポロジーを含み、容易に拡張することができます。また、MONAIの他の部分から独立しているので、既存の学習コードでネットワークを使用することができます。

MONAIには、以下のサブモジュールがあります。
- layers: 低レベルのレイヤーを定義し、次元や他の引数に基づいてPytorchやカスタムレイヤーを選択するためのファクトリーを定義する。
- blocks: ネットワークを構成する再利用可能な特定の概念を定義する中レベルのビルディングブロック
- nets: UNet、VNet、Densenetなど、一般的なアーキテクチャの完全なネットワーク定義。

ブロックとネットワークは、LayerFactoryオブジェクトをカスタムレイヤーとPyTorchレイヤーの汎用ファクトリとして使用します。

MONAIは、以下のような定義を行うブロックを提供します。
- Convolution with activation and regularization：活性化と正則化を伴うコンボリューション
- Residual units：残差ユニット
- Squeeze/excitation：スクイーズ/エキサイテーション
- Downsampling/upsampling：ダウンサンプリング・アップサンプリング
- Subpixel convolutions：サブピクセルコンボリューション


### MONAI Layersはどのように使いますか？

In [ ]:
from monai.networks.layers import Conv, Act, split_args, Pool

### Convolutionの例

[Conv](https://docs.monai.io/en/latest/networks.html#convolution) クラスは、第1引数に2つのオプションを持ちます。第2引数には、空間次元の数を `Conv[name, dimension]` のように指定する必要があります。

In [ ]:
print(Conv[Conv.CONV, 1])
print(Conv[Conv.CONV, 2])
print(Conv[Conv.CONV, 3])
print(Conv[Conv.CONVTRANS, 1])
print(Conv[Conv.CONVTRANS, 2])
print(Conv[Conv.CONVTRANS, 3])

設定されたクラスは、"バニラ "PyTorchレイヤーです。レイヤーの引数を指定することで、それらのインスタンスを作成することができました：

In [ ]:
print(Conv[Conv.CONV, 2](in_channels=1, out_channels=4, kernel_size=3))
print(Conv[Conv.CONV, 3](in_channels=1, out_channels=4, kernel_size=3))

[Act](https://docs.monai.io/en/latest/networks.html#module-monai.networks.layers.Act) クラスは空間次元の情報を必要としませんが、追加の引数をサポートしています。


In [ ]:
print(Act[Act.PRELU])
Act[Act.PRELU](num_parameters=1, init=0.1)

これらは `("prelu", {"num_parameters": 1, "init": 0.1})` のように `(type_name, arg_dict)` のtupleで完全に指定することができます：

In [ ]:
act_name, act_args = split_args(("prelu", {"num_parameters": 1, "init": 0.1}))
Act[act_name](**act_args)

## **5. これらのコンポーネントを使用して、どのようにネットワークを作成するのですか？**

### 柔軟な定義が可能なネットワーク

これらの API を使用すると、ネットワークを柔軟に定義することができます。 以下では、 `Conv`、`Act`、`Pool` を利用した `MyNetwork` というクラスを作成します。 各 Network は `__init__` と `forward` 関数が必要です。

In [ ]:
class MyNetwork(torch.nn.Module):
    
  def __init__(self, dims=3, in_channels=1, out_channels=8, kernel_size=3, pool_kernel=2, act="relu"):
    super(MyNetwork, self).__init__()
    # convolution
    self.conv = Conv[Conv.CONV, dims](in_channels, out_channels, kernel_size=kernel_size)
    # activation
    act_type, act_args = split_args(act)
    self.act = Act[act_type](**act_args)
    # pooling
    self.pool = Pool[Pool.MAX, dims](pool_kernel)
  
  def forward(self, x: torch.Tensor):
    x = self.conv(x)
    x = self.act(x)
    x = self.pool(x)
    return x

このネットワーク定義は、2Dまたは3D入力をサポートするようにインスタンス化することができ、カーネルサイズも柔軟に変更することができます。 同じアーキテクチャの設計を異なるタスクに適用するときに便利で、2D、2.5D、3Dを簡単に切り替えることができます。

MONAIのレイヤー、ブロック、ネットワークのほとんどは、`torch.nn.modules`の拡張であり、このパターンに従っています。これにより、どのようなPyTorchパイプラインとも互換性があり、ネットワーク設計にも柔軟に対応できる実装となっています。これらの微分可能なモジュールの現在のコレクションは、https://docs.monai.io/en/latest/networks.html にリストアップされています。

In [ ]:
# default network instance
default_net = MyNetwork()
print(default_net)
print(default_net(torch.ones(3, 1, 20, 20, 30)).shape)

# 2D network instance
elu_net = MyNetwork(dims=2, in_channels=3, act=("elu", {"inplace": True}))
print(elu_net)
print(elu_net(torch.ones(3, 3, 24, 24)).shape)

# 3D network instance with anisotropic kernels
sigmoid_net = MyNetwork(3, in_channels=4, kernel_size=(3, 3, 1), act="sigmoid")
print(sigmoid_net)
print(sigmoid_net(torch.ones(3, 4, 30, 30, 5)).shape)

MONAIは、以下を含めて20個以上のネットワークを提供しています：
- UNet
- VNet
- AHNet
- VGGのような回帰器、分類器、識別器、批評家
- HighResNet
- SENet

### UNet の例

8チャンネルの出力を持つ2つの隠れ層と、32チャンネルの出力を持つ最下層（ボトルネック層）からなる2D UNetネットワークを定義します。 strideの値は、最初のコンボリューションのストライドを示します。


In [ ]:
net = monai.networks.nets.UNet(
    dimensions=2,  # 2 or 3 for a 2D or 3D network
    in_channels=1,  # number of input channels
    out_channels=1,  # number of output channels
    channels=[8, 16, 32],  # channel counts for layers
    strides=[2, 2]  # strides for mid layers
)

ここでは、デフォルトのPReLUの代わりにリーキーReLUの活性化を持つ4層3D UNetを定義することにします。 今回はアクティベーション・レイヤー・ファクトリーであるActパラメータをインスタンス化し、LEAKYRELUのような既知のアクティベーション・レイヤーの名前を呼び出すことにします。 すべてのファクトリーにカスタムレイヤーを追加することができます。

In [ ]:
net = monai.networks.nets.UNet(
    dimensions=3,  
    in_channels=1,      
    out_channels=1,  
    channels=[8, 16, 32, 64],
    strides=[2, 2, 2],
    act=monai.networks.layers.Act.LEAKYRELU
)

### ワークフロー

MONAIはIgnite Engineのクラスを拡張しています。 これらのワークフローオブジェクトは、トレーニングプロセスの大部分を網羅し、デフォルトのトレーニングループとイベントへの応答メカニズムを提供します。 これは、各実験で書かなければならないコードの複雑さと量を減らすのに役立ち、完全にオプションのモジュールであり、代わりにLightningやCatalystなどの他のフレームワークを使用することができます。


## **まとめ**

MONAIデータセット、キャッシング、ネットワークについて説明しました。 以下はその主な内容です：

- MONAIデータセットは、一般的なデータセットで、lenプロパティ、getitemプロパティ、データサンプルを取得する際のオプションとして呼び出し可能なデータ変換を持ちます。
- データセットキャッシングを使用すると、データセット変換を保存して、学習を高速化することができます。 キャッシュのオプションには、CachingDataset、PersistentCaching、SmartCachingがあります。
- MONAIは、DecathlonDatasetを含む、一般的に使用されている医療画像データセットへのアクセスを提供します。
- 基本的なMONAIのレイヤー、ブロック、ネットワークを理解する。
- MONAIレイヤーを使って、柔軟なネットワークを実装し、異なるパラメータを持つ2つのUNetの例をインスタンス化します。


## **課題１**

### SmartCacheを使ったデータセットのインスタンス化

要件 
- 入力値から1を引くTransformを作成する。
- 新しいTransformを既存のSlowSquareと結合する。
- SmartCacheDatasetを置換率0.2、キャッシュ数5でインスタンス化する。
- SmartCacheを5回実行し、置換後の値を確認する。

In [ ]:
from monai.data import SmartCacheDataset
from monai.transforms import Compose, MapTransform

class MinusOne(MapTransform):
    def __init__(self, keys):
        MapTransform.__init__(self, keys)
        print(f"keys to minus from: {self.keys}")

    def __call__(self, x):
        output = {key: x[key] - 1 for key in self.keys}
        return output

smart_transform = Compose([SlowSquare(keys='data', ), MinusOne(keys='data', )])    

smart_square  = monai.data.SmartCacheDataset(
    items, 
    transform=smart_transform, 
    replace_rate = .2,
    cache_num=5
)

%time for item in smart_square: print(item)

以下は、SmartCacheを利用したものです。

In [ ]:
#Run through SmartCache replacement N-times
smart_square.start()

for i in range(5):
    print("\nCache: ", i)
    for item in smart_square: 
        print(item)
    smart_square.update_cache()
    
smart_square.shutdown()

### **課題２**

MONAI ライブラリからネットワークを即座に作成しなさい。その出力から、ネットワークにどのコンポーザブル層が使用されているかを確認しなさい。

In [ ]:
from monai.networks.nets import DenseNet121

net = DenseNet121(
    spatial_dims=2,
    in_channels=2,
    out_channels=3
)
print(net)

UNet などの一部のネットワークは、レイヤーを作成するためにテンプレート・メソッドを使用して定義されています。これらのクラスを拡張して、これらのメソッドをオーバーライドすることによって、作成されたネットワークの構成を変更する実験をしてみてください：

In [ ]:
import torch.nn as nn
from monai.networks.layers.factories import Conv
from monai.networks.nets import UNet


class MyUNet(UNet):
    def _get_down_layer(self, in_channels: int, out_channels: int, strides: int, is_top: bool) -> nn.Module:
        conv_type = Conv[Conv.CONV, self.dimensions]
        return conv_type(in_channels, out_channels, self.kernel_size)


unet = MyUNet(2, 1, 1, [2, 4, 8, 16], [2, 2, 2])
print(unet)

MyUNet(
  (model): Sequential(
    (0): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1))
    (1): SkipConnection(
      (submodule): Sequential(
        (0): Conv2d(2, 4, kernel_size=(3, 3), stride=(1, 1))
        (1): SkipConnection(
          (submodule): Sequential(
            (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
            (1): SkipConnection(
              (submodule): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
            )
            (2): Convolution(
              (conv): ConvTranspose2d(24, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
              (adn): ADN(
                (N): InstanceNorm2d(4, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
                (D): Dropout(p=0.0, inplace=False)
                (A): PReLU(num_parameters=1)
              )
            )
          )
        )
        (2): Convolution(
          (conv): ConvTranspose2d(8, 2, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)

## **Next Steps**

次のノートブックでは、MONAIのエンドツーエンドのワークフローをカバーします。

ここでカバーされている内容については、[MONAI ドキュメントページ](https://docs.monai.io/) で詳しく説明されています。 

もし、もっと例やチュートリアルをお探しなら、それ専用のレポがあります!  GitHub組織ページ](https://github.com/Project-MONAI/tutorials)でご覧いただけます。